# Serving a Keras Resnet Model

The Keras exercises are simplified as most of the concepts here are primarily taught in the [Tensorflow Estimator API notebook](./estimator_training_to_serving.ipynb)

This notebook teaches how to create a servable Resnet50 model from Keras using a pre-trained Resnet50 ImageNet model provided by the Keras library. The primary objective in this exercise is to convert the model's default input and output formats into one intended , and set its inputs field and output fields in the prediction signature definition in the Tensorflow saved_model library. (Compare this to the Estimator API, where the input signature is defined by the `serving_input_receiver_fn()` dictionary argument, and the output signature is defined by the EstimatorSpec constructor's export_outputs field in the model_fn.

See https://github.com/keras-team/keras/blob/master/keras/applications/resnet50.py for the implementation of ResNet50.

# Preamble

Import the required libraries.

In [0]:
# Import Keras libraries
import keras.applications.resnet50 as resnet50
from keras.preprocessing import image
from keras import backend as K
import numpy as np

# Import Tensorflow saved model libraries
import tensorflow as tf
from tensorflow.python.saved_model import builder as saved_model_builder
from tensorflow.python.saved_model import utils
from tensorflow.python.saved_model import tag_constants, signature_constants
from tensorflow.python.saved_model.signature_def_utils_impl import build_signature_def, predict_signature_def
from tensorflow.contrib.session_bundle import exporter

# Constants

In [0]:
_DEFAULT_IMAGE_SIZE = 224

# Setting the Output Directory

Unlike the Estimator API that automatically creates a servable version number using the unix timestamp, building a servable model directly from a tensorflow graph requires creating an explicit integer version number.

Note that if you've successfully saved the servable in a directory, trying to save another servable will fail hard. You always want to increment your version number, or otherwise delete the output directory and re-run the servable creation code.

In [0]:
VERSION_NUMBER = 1
SERVING_DIR = "keras_resnet_servable/" + str(VERSION_NUMBER)

# Build the Servable Model from Keras

Keras has a prepackaged ImageNet-trained ResNet50 model which takes in a 4d input tensor and outputs a list of class probabilities for all of the classes.

We will create a servable model whose input and output formats are identical to that provided in the [Estimator API version](./estimator_training_to_serving_solution.ipynb). Basically, the input needs is a list of jpegs, and the output needs to contain the top k classes and probabilities. We've refactored the input preprocessing and output postprocessing into helper functions.

# Helper Functions

The point of creating helper functions is two-fold:

1. Modularity: you can reuse functions in different places; for instance, a different image model or ResNet architecture can reuse functions.
2. Testability: you can unit test different parts of your code easily!

We are going to focus on building simple helper functions and performing unit tests below.

## Helper function: convert JPEG strings to Normalized 3D Tensors

The client (resnet_client.py) sends jpeg encoded images into an array of jpegs (each entry a string) to send to the server. These jpegs are all appropriately resized to 224x224x3, and do not need resizing on the server side to enter into the ResNet model. However, the ResNet50 model was trained with pixel values normalized (approximately) between -0.5 and 0.5. We will need to extract the raw 3D tensor from each jpeg string and normalize the values.

**Exercise:** Add a command in the helper function to decode the jpeg string into a 3D RGB image tensor.


In [0]:
# Preprocessing helper function similar to `resnet_training_to_serving_solution.ipynb`.

def convert_jpeg_to_image(encoded_image):
  """Preprocesses the image by subtracting out the mean from all channels.
  Args:
    image: A jpeg-formatted byte stream represented as a string.
  Returns:
    A 3d tensor of image pixels normalized for the Keras ResNet50 model.
      The canned ResNet50 pretrained model was trained after running
      keras.applications.resnet50.preprocess_input in 'caffe' mode, which
      flips the RGB channels and centers the pixel around the mean [103.939, 116.779, 123.68].
      There is no normalizing on the range.
  """
  image = tf.image.decode_jpeg(encoded_image, channels=3)
  image = tf.to_float(image)
  image = resnet50.preprocess_input(image)  
  return image

### Unit test the helper function

Unit testing is discussed in more detail in [the Estimator exercise](./estimator_training_to_serving.ipynb) and will be omitted here.

**Exercise:** Construct a tensorflow unit test graph for the preprocessing function.

**Hint:** See the [Estimator notebook](./estimator_training_to_serving.ipynb).

In [0]:
# Defining input test graph nodes: only needs to be run once!
test_jpeg = tf.placeholder(dtype=tf.string, shape=[], name='test_jpeg')  # A placeholder for a single string, which is a dimensionless (0D) tensor.
test_decoded_tensor = convert_jpeg_to_image(test_jpeg)  # Output node, which returns a 3D tensor after processing.

# Print the graph elements to check shapes. ? indicates that Tensorflow does not know the length of those dimensions.
print(test_jpeg)
print(test_decoded_tensor)

In [0]:
# Validate the result of the function using a sample image client/cat_sample.jpg
ERROR_TOLERANCE = 1e-4

with open("client/cat_sample.jpg", "rb") as imageFile:
    jpeg_str = imageFile.read()
    with tf.Session() as sess:
        result = sess.run(test_decoded_tensor, feed_dict={test_jpeg: jpeg_str})
        assert result.shape == (224, 224, 3)
        # TODO: Replace with assert statements to check max and min normalized pixel values
        assert result.max() <= 255.0 - 103.939 + ERROR_TOLERANCE # Max pixel value after subtracting mean
        assert result.min() >= -123.68 - ERROR_TOLERANCE # Min pixel value after subtracting mean
        print('Hooray! JPEG decoding test passed!')

## Helper Function: Preprocessing Server Input

The server receives a client request in the form of a dictionary {'images': tensor_of_jpeg_encoded_strings}, which must be preprocessed into a 4D tensor before feeding into the Keras ResNet50 model.

**Exercise**: You will need to modify the input to the Keras Model to be compliant with [the ResNet client](./client/resnet_client.py). Using tf.map_fn and convert_jpeg_to_image, fill in the missing line (marked ???) to convert the client request into an array of 3D floating-point, preprocessed tensor. The following lines stack and reshape this array into a 4D tensor.

**Useful References:**
* [tf.map_fn](https://www.tensorflow.org/api_docs/python/tf/map_fn)
* [tf.DType](https://www.tensorflow.org/api_docs/python/tf/DType)

In [0]:
def preprocess_input(jpeg_tensor):
    processed_images = tf.map_fn(convert_jpeg_to_image, jpeg_tensor, dtype=tf.float32)  # Convert list of JPEGs to a list of tensors
    processed_images = tf.stack(processed_images)  # Convert list of tensors to tensor of tensors
    processed_images = tf.reshape(tensor=processed_images,  # Reshape to ensure TF graph knows the final dimensions
                                shape=[-1, _DEFAULT_IMAGE_SIZE, _DEFAULT_IMAGE_SIZE, 3])
    return processed_images

### Unit Test the Input Preprocessing Helper Function

**Exercise**: Construct a tensorflow unit test graph for the input function.

**Hint:** the input node test_jpeg_tensor should be a [tf.placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder). You need to define the `shape` parameter in tf.placeholder. `None` inside an array indicates that the length can vary along that dimension.

In [0]:
# Build a Test Input Preprocessing Network: only needs to be run once!
test_jpeg_tensor = tf.placeholder(dtype=tf.string, shape=[None], name='test_jpeg_tensor')  # A placeholder for a single string, which is a dimensionless (0D) tensor.
test_processed_images = preprocess_input(test_jpeg_tensor)  # Output node, which returns a 3D tensor after processing.

# Print the graph elements to check shapes. ? indicates that Tensorflow does not know the length of those dimensions.
print(test_jpeg_tensor)
print(test_processed_images)

In [0]:
# Run test network using a sample image client/cat_sample.jpg

with open("client/cat_sample.jpg", "rb") as imageFile:
    jpeg_str = imageFile.read()
    with tf.Session() as sess:
        result = sess.run(test_processed_images, feed_dict={test_jpeg_tensor: np.array([jpeg_str, jpeg_str])})  # Duplicate for length 2 array
        assert result.shape == (2, 224, 224, 3)  # 4D tensor with first dimension length 2, since we have 2 images
        # TODO: add a test for min and max normalized pixel values
        assert result.max() <= 255.0 - 103.939 + ERROR_TOLERANCE  # Normalized
        assert result.min() >= -123.68 - ERROR_TOLERANCE  # Normalized
        # TODO: add a test to verify that the resulting tensor for image 0 and image 1 are identical.
        assert result[0].all() == result[1].all()
        print('Hooray! Input unit test succeeded!')

## Helper Function: Postprocess Server Output

**Exercise:** The Keras model returns a 1D tensor of probabilities for each class. We want to wrote a postprocess_output() that returns only the top k classes and probabilities.

**Useful References:**
* [tf.nn.top_k](https://www.tensorflow.org/api_docs/python/tf/nn/top_k)

In [0]:
TOP_K = 5

def postprocess_output(model_output):
    '''Return top k classes and probabilities.'''
    top_k_probs, top_k_classes = tf.nn.top_k(model_output, k=TOP_K)
    return {'classes': top_k_classes, 'probabilities': top_k_probs}


### Unit Test the Output Postprocessing Helper Function

**Exercise:** Fill in the shape field for the model output, which should be a tensor of probabilities.

**Hint:** how many image classes are there?

In [0]:
# Build Test Output Postprocessing Network: only needs to be run once!
test_model_output = tf.placeholder(dtype=tf.float32, shape=[1001], name='test_logits_tensor')
test_prediction_output = postprocess_output(test_model_output)

# Print the graph elements to check shapes.
print(test_model_output)
print(test_prediction_output)

In [0]:
# Import numpy testing framework for float comparisons
import numpy.testing as npt

# Run test network
# Input a tensor with clear winners, and perform checks

model_probs = np.ones(1001)
model_probs[2] = 2.5
model_probs[5] = 3.5
model_probs[10] = 4
model_probs[49] = 3
model_probs[998] = 2
TOTAL_WEIGHT = np.sum(model_probs)
model_probs = model_probs / TOTAL_WEIGHT

with tf.Session() as sess:
    result = sess.run(test_prediction_output, {test_model_output: model_probs})
    classes = result['classes']
    probs = result['probabilities']
    # Check values
    assert len(probs) == 5
    npt.assert_almost_equal(probs[0], model_probs[10])
    npt.assert_almost_equal(probs[1], model_probs[5])
    npt.assert_almost_equal(probs[2], model_probs[49])
    npt.assert_almost_equal(probs[3], model_probs[2])
    npt.assert_almost_equal(probs[4], model_probs[998])
    assert len(classes) == 5
    assert classes[0] == 10
    assert classes[1] == 5
    assert classes[2] == 49
    assert classes[3] == 2
    assert classes[4] == 998
    print('Hooray! Output unit test succeeded!')

# Load the Keras Model and Build the Graph

The Keras Model uses Tensorflow as its backend, and therefore its inputs and outputs can be treated as elements of a Tensorflow graph. In other words, you can provide an input that is a Tensorflow tensor, and read the model output like a Tensorflow tensor!

**Exercise**: Build the end to end network by filling in the TODOs below.

**Useful References**:
* [Keras ResNet50 API](https://www.tensorflow.org/api_docs/python/tf/keras/applications/ResNet50)
* [Keras Model class API](https://faroit.github.io/keras-docs/1.2.2/models/model/): ResNet50 model inherits this class.

In [0]:
# TODO: Create a placeholder for your arbitrary-length 1D Tensor of JPEG strings
images = tf.placeholder(dtype=tf.string, shape=[None])

# TODO: Call preprocess_input to return processed_images
processed_images = preprocess_input(images)

# Load (and download if missing) the ResNet50 Keras Model (may take a while to run)
# TODO: Use processed_images as input
model = resnet50.ResNet50(input_tensor=processed_images)
# Rename the model to 'resnet' for serving
model.name = 'resnet'

# TODO: Call postprocess_output on the output of the model to create predictions to send back to the client
predictions = postprocess_output(model.output)

# Creating the Input-Output Signature

**Exercise:** The final step to creating a servable model is to define the end-to-end input and output API. Edit the inputs and outputs parameters to predict_signature_def below to ensure that the signature correctly handles client request. The inputs parameter should be a dictionary {'images': tensor_of_strings}, and the outputs parameter a dictionary {'classes': tensor_of_top_k_classes, 'probabilities': tensor_of_top_k_probs}.

In [0]:
# Create a saved model builder as an endpoint to dataflow execution
builder = saved_model_builder.SavedModelBuilder(SERVING_DIR)

# TODO: set the inputs and outputs parameters in predict_signature_def()
signature = predict_signature_def(inputs={'images': images},
                                  outputs=predictions)

# Export the Servable Model

In [0]:
with K.get_session() as sess:
    builder.add_meta_graph_and_variables(sess=sess,
                                         tags=[tag_constants.SERVING],
                                         signature_def_map={'predict': signature})
    builder.save()